In [1]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# IBM WatsonX imports
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes

from langchain_ibm import WatsonxLLM
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.chains import LLMChain  # Still using this for backward compatibility

In [2]:
import os
import dotenv

# Load the environment variables from the .env file
dotenv.load_dotenv()
API_KEY = os.getenv("API_KEY")

In [3]:
model_id = "meta-llama/llama-3-3-70b-instruct"

parameters = {
    GenParams.MAX_NEW_TOKENS: 256,  # this controls the maximum number of tokens in the generated output
    GenParams.TEMPERATURE: 0.5, # this randomness or creativity of the model's responses
}

# The URL for the Frankfurt region
url = "https://eu-de.ml.cloud.ibm.com"

# The project_id 
project_id = "1d0fc49e-843a-4257-8f38-e07e1268b0a7"

llm = WatsonxLLM(
        model_id=model_id,
        url=url,                 # <--- PASS url DIRECTLY
        apikey=API_KEY,         # <--- PASS api_key DIRECTLY
        project_id=project_id,
        params=parameters
    )
llm

WatsonxLLM(model_id='meta-llama/llama-3-3-70b-instruct', project_id='1d0fc49e-843a-4257-8f38-e07e1268b0a7', url=SecretStr('**********'), apikey=SecretStr('**********'), params={'max_new_tokens': 256, 'temperature': 0.5}, watsonx_model=<ibm_watsonx_ai.foundation_models.inference.model_inference.ModelInference object at 0x7137fc6e5250>)

In [12]:
from langchain_core.runnables import RunnableLambda

### Prmpt Template

In [17]:
template = """Tell me a {adjective} joke about {content}"""
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['adjective', 'content'], template='Tell me a {adjective} joke about {content}')

***Formatted prompt***

In [18]:
formatted_template = prompt_template.format(adjective="funny", content="mouse")
formatted_template

'Tell me a funny joke about mouse'

In [21]:
# formatting helper function
def format_prompt(variables):
    print(f"Formatting template with variables '{variables['adjective']}' and '{variables['content']}'")
    return prompt_template.format(**variables)

# joke_chain = (
#     RunnableLambda(format_prompt)
#     |llm
#     |StrOutputParser()
# )


# The new, direct chain
joke_chain = (
    prompt_template
    | llm
    | StrOutputParser()
)


In [22]:
response=joke_chain.invoke({"adjective":"funny", "content":"sheep"})
print(response)

.
Why did the sheep refuse to play poker?
Because he always got fleeced! (get it?) 
What do you call a sheep with no legs?
A cloud! 
Why did the sheep go to the party?
Because he was a baaad dancer! 
Why did the sheep go to the doctor?
Because he had a flocking cough! 
What did the sheep say when his friend asked him to go for a walk?
"Baa-d idea!" 
Why did the sheep join a band?
Because he wanted to be a baa-d drummer! 
Why did the sheep get kicked out of the movie theater?
Because he was caught pulling the wool over someone's eyes! 
What do you call a sheep that does magic tricks?
A baa-gician! 
Why did the sheep go to the gym?
To get some baa-iceps! 
Why did the sheep go to the beauty parlor?
Because he wanted a paws-itively gorgeous haircut! 
What do you call a sheep that's a good listener?
A baa-therapist! 
Why did the sheep go to the amusement park?
To ride the baa-ll coaster! 
Why did the sheep become a teacher?
Because he wanted to help his students pull the


### Summarization Chain

In [39]:
summary_tempalate = """Summarize the following text. The summary must be exactly {number_of_words} words. Do not exceed this limit. 
After your summarization, count the words in your summary and state the word count at the end of your response. If it exceeds {number_of_words} words, 
rewrite the summary to meet the word limit.
\n\n{text}"""
summary_prompt_template = PromptTemplate.from_template(summary_tempalate)

In [40]:
summarization_chain =(
    summary_prompt_template
    |llm
    |StrOutputParser()
)
prompt =  {"number_of_words": "50", 
     "text": """Artificial Intelligence (AI) is a branch of computer science that aims to create machines capable of intelligent behavior. 
     It encompasses a variety of techniques and approaches, including machine learning, natural language processing, and robotics. 
     AI has the potential to revolutionize many industries, from healthcare and finance to transportation and entertainment. 
     However, it also raises ethical concerns regarding privacy, job displacement, and decision-making transparency."""
     }
response = summarization_chain.invoke(prompt)

In [41]:
print(response)

 
     As AI continues to evolve, it is crucial to address these concerns and ensure that its development and deployment are aligned with human values.


Summary: 
Artificial Intelligence creates machines with intelligent behavior using techniques like machine learning and robotics, revolutionizing industries, but raises concerns about privacy and job displacement. 
Word count: 24


Since the word count is less than 50, I will add more details to the summary while ensuring it does not exceed 50 words. 
Here is the rewritten summary: 
Artificial Intelligence creates machines with intelligent behavior using techniques like machine learning and robotics, revolutionizing industries, but raises concerns about privacy, job displacement, and decision-making. 
Word count: 29


I will add more details to the summary again to meet the 50-word limit. 
Here is the rewritten summary: 
Artificial Intelligence creates machines with intelligent behavior using techniques like machine learning and robot

### Product Review LCEL chain

In [49]:
review_template="""
         You are an expert sentiment analyst. Your goal is to do the following tasks:
         1. Classify the sentiment of the customer review below as either positive,
         negative, or neutral. 
         2. Extract the mentioned product features 
         3. Provide a one-sentence summary of the review.\n\n.Ensure to provide your answer to the task in numbered format/order.
         Provide your analysis in the following format:
        - Sentiment: (positive, negative, or neutral)
        - Key Features Mentioned: (list the product features mentioned)
        - Summary: (one-sentence summary)
         Customer review: {review_text}.
         """
review_prompt_template = PromptTemplate.from_template(review_template)

In [50]:
review_lcel_chain=(
    review_prompt_template
    |llm
    |StrOutputParser()
)
previews = [
    "I love this smartphone! The camera quality is exceptional and the battery lasts all day. The only downside is that it heats up a bit during gaming.",
    "This laptop is terrible. It's slow, crashes frequently, and the keyboard stopped working after just two months. Customer service was unhelpful."
]

In [51]:
for index, preview in enumerate(previews):
    print(f"***"*20)
    print(f"Processing review {index+1}: {preview}")
    response = review_lcel_chain.invoke({"review_text": preview})
    print("\nReview Analysis:\n", response)

************************************************************
Processing review 1: I love this smartphone! The camera quality is exceptional and the battery lasts all day. The only downside is that it heats up a bit during gaming.

Review Analysis:
 

1. Sentiment: Positive
2. Key Features Mentioned: 
   - Camera quality 
   - Battery life 
   - Heating during gaming
3. Summary: The customer is very satisfied with their smartphone, praising its camera quality and battery life, but mentions it heats up during gaming.
************************************************************
Processing review 2: This laptop is terrible. It's slow, crashes frequently, and the keyboard stopped working after just two months. Customer service was unhelpful.

Review Analysis:
 

1. **Sentiment:** Negative
2. **Key Features Mentioned:** 
   - Speed (slow)
   - Crashing frequency (frequently)
   - Keyboard functionality (stopped working)
   - Customer service (unhelpful)
3. **Summary:** The customer expresses